In [3]:
import time
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy.matlib as npm
from scipy import signal
import pandas as pd

In [4]:
class radar1:
    def __init__(self):
        self.fc = 60*(10**9)
        self.fs = 2.64*(10**9)
        self.L = 6272
        self.PRI = 2*(10**-6)
        self.PTM_len = 100
        self.c = 3*(10**8)
        self.ts = 1/self.fs
        self.noise = 0
        self.Lambda = self.c/self.fc
        self.windowsum = 0
        self.ptx = 0
        self.gt = 0
        self.gr = 0
        self.ptx_linear = 10**(self.ptx/10)*(10**-3)
        self.gt_linear = 10**(self.gt/10)
        self.gr_linear = 10**(self.gr/10)
        self.radar_coeff = (np.sqrt(self.ptx_linear * self.gt_linear * self.gr_linear) * self.Lambda) / (4 * np.pi)

In [5]:
class radar2:
    def __init__(self):
        self.fc = 60*(10**9)
        self.fs = 1.76*(10**9)
        self.L = 512
        self.PRI = 2*(10**-6)
        self.c = 3*(10**8)
        self.ts = 1/self.fs
        self.noise = 1
        self.PTM_len = 100
        self.Lambda = self.c/self.fc
        self.dfd = 121
        self.im_fdmax = 12*(10**3)
        self.im_fdmin = -12*(10**3)
        self.snr = 45 # Variable SNR
        self.snr_linear = 10**(self.snr/10)

In [6]:
class target:
    def __init__(self, range_t, doppler_velocity, rcs_amp):
        self.range = range_t
        self.doppler_velocity = doppler_velocity
        self.rcs_amp = rcs_amp
        self.n_bodypart = 1

In [7]:
class antenna:
    def __init__(self, Lambda, Nelements, DOA):
        self.d_m = Lambda * 0.5
        self.Nelements = Nelements
        self.spacing = np.arange(0, (self.Nelements-1) * self.d_m, self.d_m)
        self.DOA = DOA # Angle for azimuth

In [26]:
def get_received_signal(target, radar1, xmat1, fs, fc, antenna, n_frame, radar2):
    print("Generating RX Signal + Noise")
    Lvec = radar1.ts * (np.arange(0,radar1.L))
    Mvec = radar1.PRI * np.arange(0,radar1.PTM_len)
    Larray = np.transpose(npm.repmat(Lvec, radar1.PTM_len, 1))
    Marray = npm.repmat(Mvec, radar1.L, 1)
    time_array = Larray + Marray

    w = np.hamming(radar1.PTM_len)
    wmat = npm.repmat(np.transpose(w), radar1.L, 1)
    window_sum = np.sum(w)

    # Channel Modelling
    radar1.ptx = 30
    radar1.gt = 0
    radar1.gr = 0

    rxmat = np.zeros((2*radar1.L-1, radar1.PTM_len))
    range_to_delay_symbols = np.floor(((2*target.range)/radar1.c)/radar1.ts)

    for nb in range(0,target.n_bodypart):
        channel = np.zeros((radar1.L,1))
        velvec = target.doppler_velocity
        Doppler_ele = 2*np.pi*velvec/radar1.Lambda
        Doppler_Delay = np.exp(1j*(Doppler_ele*time_array))

        temp = np.multiply(np.multiply(wmat,xmat1),Doppler_Delay)
        # print(temp)
        channel[int(range_to_delay_symbols)] = 1

        # amp = (target.rcs_amp[n_frame][nb])/(target.r[n_frame][nb]**2)
        amp = (target.rcs_amp)/(target.range**2)
        rxmat_temp = (amp**1000)*signal.convolve2d(channel,temp)
        rxmat = rxmat + rxmat_temp
    rxmat1 = rxmat/target.n_bodypart
    rxmat2 = rxmat1 * radar1.radar_coeff
    return rxmat2.real


In [9]:
radar1_in = radar1()
radar2_in = radar2()
doppler_axis = np.arange(radar2_in.im_fdmin, radar2_in.im_fdmax + radar2_in.dfd, radar2_in.dfd)
velocity_axis = doppler_axis * radar2_in.Lambda/2
delay_axis = np.arange(-radar2_in.L + 1, radar2_in.L)
range_axis = ((radar2_in.c * radar2_in.ts) / 2) * delay_axis

In [19]:
range_t = 15
velocity = 4
rcs_amp = 1
t1 = target(range_t, velocity, rcs_amp)

Nu_elements = 10 # Variable Number of Antenna
DOA_angle = 45 # Angle of Target
antenna_param = antenna(radar2_in.Lambda, Nu_elements, DOA_angle)

In [27]:
# Load XMAT
xmat1 = sio.loadmat("./data/jrc_tx.mat")["xmat1"]
n_frame = 1

rxmat2 = get_received_signal(t1, 
                                radar1_in, 
                                xmat1, 
                                radar1_in.fs, 
                                radar1_in.fc, 
                                antenna_param, 
                                n_frame, 
                                radar2_in)

Generating RX Signal + Noise
0.0044444444444444444
